In [1]:
import json

In [2]:
!ls ../multihop_dense_retrieval/data/hotpot/hotpot_qas_val.json

../multihop_dense_retrieval/data/hotpot/hotpot_qas_val.json


In [3]:
SPLIT = "dev"

In [5]:
fp_hotpot_mdr = '../multihop_dense_retrieval/data/hotpot/hotpot_dev_with_neg_v0.json'
fp_hotpot_original = '../multihop_dense_retrieval/data/hotpot/hotpot_dev_fullwiki_v1.json'
fp_hotpot_retr = '../multihop_dense_retrieval/data/hotpot/dev_retrieval_top100_sp.json'

hotpot_mdr = []
with open(fp_hotpot_mdr, 'r') as handle:
    lines = handle.readlines()
    for l in lines:
        hotpot_mdr.append(json.loads(l))
    
with open(fp_hotpot_original, 'r') as handle:
    hotpot_orig = json.load(handle)
    
hotpot_retr = []
with open(fp_hotpot_retr, 'r') as handle:
    lines = handle.readlines()
    for l in lines:
        hotpot_retr.append(json.loads(l))

In [10]:
hotpot_retr[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'type': 'comparison',
 'sp': [{'title': 'Scott Derrickson',
   'sents': ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
    ' He lives in Los Angeles, California.',
    ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."'],
   'sp_sent_ids': [0]},
  {'title': 'Ed Wood',
   'sents': ['Edward Davis Wood Jr. (October 10, 1924\xa0– December 10, 1978) was an American filmmaker, actor, writer, producer, and director.'],
   'sp_sent_ids': [0]}],
 'answer': ['yes'],
 'candidate_chains': [[{'title': 'Scott Derrickson',
    'sents': ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
     ' He lives in Los Angeles, California.',
     ' He is best known 

In [11]:
hotpot_orig[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'answer': 'yes',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]],
 'context': [['Adam Collis',
   ['Adam Collis is an American filmmaker and actor.',
    ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
    ' He also studied cinema at the University of Southern California from 1991 to 1997.',
    ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
    ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".']],
  ['Ed Wood (film)',
   ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lug

In [12]:
def extract_sentences(sup_idx_list, context_list, doc_title=None):
    return_obj = []
    sup = {}
    for s in sup_idx_list:
        if s[0] not in sup:
            sup[s[0]] = []
        sup[s[0]].append(s[1])
    for c in context_list:
        if c['title'] not in sup:
            continue
        if doc_title is not None and c['title'] != doc_title:
            continue
        for i in sup[c['title']]:
            try:
                return_obj.append(c['sents'][i])
            except:
                print("IndexError: ignoring")
                print(c['title'])
                print(c['sents'])
                print(sup[c['title']])
    return return_obj

In [13]:
output = []

for i in range(len(hotpot_orig)):
    orig_obj = hotpot_orig[i]
    mdr_obj = hotpot_mdr[i]
    retr_obj = hotpot_retr[i]
    q_obj = {
        "id": orig_obj["_id"],
        "question": orig_obj["question"],
        "level": orig_obj["level"],
        "type": orig_obj["type"],
        "answers": [],
        "final_answer": orig_obj["answer"],
    }
    if orig_obj["type"] == "comparison":
        # Fetch all SUP sentences into "answers"
        q_obj["answers"] = extract_sentences(orig_obj["supporting_facts"], retr_obj["sp"])
    else:
        # Fetch only the first-hop SUP sentences into "answers"
        for sup in orig_obj["supporting_facts"]:
            if sup[0] != mdr_obj["bridge"]:
                q_obj["answers"] = extract_sentences(orig_obj["supporting_facts"], retr_obj["sp"], sup[0])
                break
    output.append(q_obj)

IndexError: ignoring
Jimmy Butler (basketball)
['Jimmy Butler III (born September 14, 1989) is an American professional basketball player for the Minnesota Timberwolves of the National Basketball Association (NBA).', ' Born in Houston, Butler grew up in Tomball, Texas, and played college basketball for Tyler Junior College and Marquette University.', ' He was drafted with the 30th overall pick in the 2011 NBA draft by the Chicago Bulls.', ' He is a three-time NBA All-Star and a three-time NBA All-Defensive Team honoree, and was named to his first All-NBA Team in 2017.', ' In 2015, he was named the NBA Most Improved Player.']
[0, 902]


In [14]:
# Store results to disk

output_fp = f"../multihop_dense_retrieval/data/hotpot/hotpot_{SPLIT}_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": output}, fp, indent=4)

In [16]:
output[1]

{'id': '5a8c7595554299585d9e36b6',
 'question': 'What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?',
 'level': 'hard',
 'type': 'bridge',
 'answers': ['Kiss and Tell is a 1945 American comedy film starring then 17-year-old Shirley Temple as Corliss Archer.'],
 'final_answer': 'Chief of Protocol'}